In [1]:
import sqlalchemy
import pandas as pd
import os
import datetime
from tqdm.notebook import tqdm as tqdm_notebook
#import mmh3
import seaborn as sns

from sqlalchemy import create_engine
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas_gbq

from clickhouse_sqlalchemy import make_session
from urllib.parse import quote

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style("darkgrid")
sns.set(rc={'figure.figsize':(12,7)})

import pickle
import lightgbm

import pandahouse

In [2]:
#креды для подключения к КХ
connection_kh = {
            'host': 'http://clickhouse-02.lenta.tech:8123',
            'password': 'Gnjrw1HhkByi',
            'user': 'a_al_kosheleva',
            'database':'dwh_stage'
            }

In [2]:
#берем вчерашнюю дату и дату N дней назад, в нашем случае 90
from datetime import datetime, timedelta
yesterday = datetime.now() - timedelta(1)
three_months_ago = datetime.now() - timedelta(91)

first_date = datetime.strftime(three_months_ago, '%Y-%m-%d')
last_date = datetime.strftime(yesterday, '%Y-%m-%d')

In [4]:
#выгружаем активных юзеров из bq за определенный день, за вчера
sql = f'''SELECT distinct user_id 
FROM dwh_datamart.daily_aud_aggregate_app
WHERE event_date = '{last_date}' and any_active > 0
and user_id is not null and user_id != 'null'
'''
buyers = pandahouse.read_clickhouse(sql, connection=connection_kh)

In [3]:
#креды для выгрузки из двх
password = 'dimVuidBun1onCub'
login = 'a.al.kosheleva'

def postre_auth(login, password):
    postgresql_config = {
    'postgresql_prefix' : 'postgresql'
    , 'postgresql_host'   : '10.3.26.92' 
    , 'postgresql_user'   : f'{login}'
    , 'postgresql_pass'   : f'{password}'
    , 'postgresql_port'   : '5432'
    , 'postgresql_db'     : 'dwh'    
    }

    postgresql_connection_string  = "{prefix}://{user}:{passw}@{host}:{port}/{db}".format(
    prefix  =   postgresql_config['postgresql_prefix']    , host  =   postgresql_config['postgresql_host']
    , user  =   postgresql_config['postgresql_user']      , passw =   postgresql_config['postgresql_pass']
    , port  =   postgresql_config['postgresql_port']      , db    =   postgresql_config['postgresql_db']
    )
    postgresql_engine = sqlalchemy.create_engine(postgresql_connection_string)
    postgresql_conn   = postgresql_engine.connect()
    return postgresql_engine, postgresql_conn

postrgre_engine, postrge_conn = postre_auth(login, password)

In [6]:
buyers_list = tuple(buyers.user_id.to_list())

In [7]:
#отбираем только старых покупателей - у кого первый заказ был 90 дней назад
old_users = f'''
select distinct buyer_id
from dwh_datamart.retention
where date(first_order) <= '{first_date}'
and buyer_id IN {buyers_list}
'''

In [8]:
old_users_list = pd.read_sql(old_users, postrge_conn)

In [9]:
buyers_list_fin = old_users_list.buyer_id.to_list()

In [10]:
#выгрузка метрик без категорий - чек и т.д.
financial_data = f'''
with
orders AS (
select distinct buyer_id, pav_order_id, bill_amount
from dwh_datamart.ops_metrics
where date(created) between '{first_date}' AND '{last_date}'
and flag_cancel = 0 and order_source IN ('iOS', 'Android')
),

pcodes AS (
select distinct pav_order_id, action_id
from dwh_stage.utk_sess_utk_promocode_buyer_using
where date(created) between '{first_date}' AND '{last_date}'
),

orders_with_pcodes AS (
select buyer_id, o.pav_order_id, bill_amount,
       case when action_id is null then  0 else 1 end as pcode
from orders o left join pcodes p on o.pav_order_id = p.pav_order_id
)

select buyer_id,
sum(bill_amount)/count(distinct pav_order_id) as avg_check,
sum(bill_amount) as revenue,
count(distinct pav_order_id) as frequency, round(sum(pcode)*1.00/count(distinct pav_order_id),2) as pcode_orders_percent
from orders_with_pcodes
group by 1;
'''

In [6]:
#выгрузка по категориям
categories_data = f'''
with cats AS (
select
  substr(mat.material,13,18)  original_id,
  wgh._0rpa_wgh2___t          cat_name
from dwh_stage.hadoop_bi0_pmaterial mat
inner join dwh_stage.hadoop_bic_rpa_wgh wgh
on mat.rpa_wgh4 = wgh._0rpa_wgh4
where rpa_wgh2 != '99'
and wgh._0rpa_wgh2___t != 'Резервная группа'
group by 1,2),

orders AS (
select distinct buyer_id, pav_order_id
from dwh_datamart.ops_metrics
where date(created) between '{first_date}' AND '{last_date}'
and flag_cancel = 0 and order_source IN ('iOS', 'Android')
),

t1 as (
select
  buyer_id,
  cat_name,
  count_courier*zit.denominator/coalesce(zit.numerator, 1) as number,
  sum(count_courier*zit.denominator/coalesce(zit.numerator, 1)) over (partition by buyer_id) as count_items_total
from orders z
inner join dwh_stage.utk_sess_zakaz_items zit
on zit.pav_order_id = z.pav_order_id
left join cats c on zit.original_id = c.original_id
where zit.numerator != 0
)


select buyer_id, cat_name, sum(number)*1.00/max(count_items_total) percent_cat
from t1
where count_items_total > 0
group by 1,2
'''

In [12]:
financial_df_first = pd.read_sql(financial_data, postrge_conn)

In [7]:
categories_df_first = pd.read_sql(categories_data, postrge_conn)

In [14]:
#по нужным нам id фильтруем уже в питоне, т.к. если делать это при выгрузке ее время увеличивается в разы
financial_df = pd.DataFrame(financial_df_first.query("buyer_id == @buyers_list_fin"))
categories_df = pd.DataFrame(categories_df_first.query("buyer_id == @buyers_list_fin"))

In [15]:
#соединяем 2 датафрейма
categories_df_pivot = pd.pivot_table(categories_df, values='percent_cat', index=['buyer_id'],
                    columns=['cat_name'], aggfunc=np.sum).reset_index()
final_df = categories_df_pivot.merge(financial_df, on = 'buyer_id', how = 'inner')
final_df = final_df.fillna(0)

In [16]:
final_df_sample = final_df

In [17]:
final_df_sample.reset_index(drop= True , inplace= True)

In [18]:
final_df_sample.head()

,buyer_id,Автотовары,Аксессуары и галантерея,"Аудио, видео, цифровая техника и аксессу",Багаж,Бакалея,Безалкогольные напитки,Бытовая техника,"Бытовая химия, хозяйственные товары",Вино,...,Традиционная молочная продукция,Фрукты,Функциональное питание,Шоколадные кондитерские изделия,Электротовары,Яйцо,avg_check,revenue,frequency,pcode_orders_percent
0,94578440,0.0,0.0,0.0,0.0,0.066430,0.000000,0.0,0.199291,0.0,...,0.000000,0.000000,0.00000,0.000000,0.0,0.000000,2428.770000,2428.77,1,1.00
1,94578475,0.0,0.0,0.0,0.0,0.000000,0.027892,0.0,0.027892,0.0,...,0.018595,0.108122,0.00000,0.027892,0.0,0.018595,4828.630000,9657.26,2,1.00
2,94578547,0.0,0.0,0.0,0.0,0.017390,0.000000,0.0,0.043474,0.0,...,0.295626,0.017372,0.01739,0.000000,0.0,0.026085,1373.406667,8240.44,6,0.83
3,94578644,0.0,0.0,0.0,0.0,0.033402,0.322887,0.0,0.000000,0.0,...,0.011134,0.079649,0.00000,0.000000,0.0,0.033402,1670.076000,8350.38,5,0.40
4,94578760,0.0,0.0,0.0,0.0,0.058634,0.011727,0.0,0.000000,0.0,...,0.082087,0.103348,0.00000,0.070360,0.0,0.000000,4720.685000,9441.37,2,1.00


In [19]:
#логарифмирование данных
def get_log_df(data):
    def log_transform(v): 
        res = np.log(v+1)
        return res
    
    data_tmp = data.copy()
    for el in ['Бакалея',
       'Безалкогольные напитки',
       'Бытовая химия, хозяйственные товары', 'Гигиенические средства',
       'Деликатесы из рыбы и морепродуктов', 'Детская гигиена', 'Детское питание',
       'Живые горшечные цветы, срезка', 'Закуски к аперитивам',
       'Замороженная рыба', 'Канцелярия и печатная продукция',
       'Консервы', 'Косметика и аксессуары', 'Кофе, какао, чай',
       'Кулинария собственного производства',
       'Майонезы, Закуски, Готовые Блюда охл', 
       'Масложировая продукция', 'Молочные продукты д/детского питания',
       'Морепродукты глубокой заморозки', 
       'Мясо охлажденное', 'Не шоколадные кондитерские изделия', 'Овощи',
       'Охлажденная и переработанная рыбная прод',
       'П\ф и другие продукты глубокой заморзк',
       'Пекарня промышленное производство', 'Пекарня собственное производство',
       'Посуда столовая, товары для дома',
       'Продукты мясной переработки',  'Птица охлажденная',
       'Современная молочная продукция', 
       'Средства персонального ухода', 'Сыры', 
       'Товары для животных', 'Товары для приготовления', 'Товары для сада',
       'Товары для уборки и хранения', 
       'Традиционная молочная продукция', 'Фрукты', 'Функциональное питание',
       'Шоколадные кондитерские изделия', 'Яйцо']:
        data_tmp[el] = data_tmp[el].apply(lambda x: log_transform(x))
    
    return data_tmp[['Бакалея',
       'Безалкогольные напитки',
       'Бытовая химия, хозяйственные товары', 'Гигиенические средства',
       'Деликатесы из рыбы и морепродуктов', 'Детская гигиена', 'Детское питание',
       'Живые горшечные цветы, срезка', 'Закуски к аперитивам',
       'Замороженная рыба', 'Канцелярия и печатная продукция',
       'Консервы', 'Косметика и аксессуары', 'Кофе, какао, чай',
       'Кулинария собственного производства',
       'Майонезы, Закуски, Готовые Блюда охл', 
       'Масложировая продукция', 'Молочные продукты д/детского питания',
       'Морепродукты глубокой заморозки', 
       'Мясо охлажденное', 'Не шоколадные кондитерские изделия', 'Овощи',
       'Охлажденная и переработанная рыбная прод',
       'П\ф и другие продукты глубокой заморзк',
       'Пекарня промышленное производство', 'Пекарня собственное производство',
       'Посуда столовая, товары для дома',
       'Продукты мясной переработки',  'Птица охлажденная',
       'Современная молочная продукция', 
       'Средства персонального ухода', 'Сыры', 
       'Товары для животных', 'Товары для приготовления', 'Товары для сада',
       'Товары для уборки и хранения', 
       'Традиционная молочная продукция', 'Фрукты', 'Функциональное питание',
       'Шоколадные кондитерские изделия', 'Яйцо']].copy()

In [20]:
df_cl_pred = get_log_df(final_df_sample.copy())

In [21]:
sc = pickle.load(open("sc_stb_may24_2.dat", "rb"))

In [22]:
df_cl = pd.DataFrame(sc.fit_transform(df_cl_pred))

In [23]:
df_cl.columns = df_cl_pred.columns

In [24]:
df_cl.head()

,Бакалея,Безалкогольные напитки,"Бытовая химия, хозяйственные товары",Гигиенические средства,Деликатесы из рыбы и морепродуктов,Детская гигиена,Детское питание,"Живые горшечные цветы, срезка",Закуски к аперитивам,Замороженная рыба,...,Сыры,Товары для животных,Товары для приготовления,Товары для сада,Товары для уборки и хранения,Традиционная молочная продукция,Фрукты,Функциональное питание,Шоколадные кондитерские изделия,Яйцо
0,-0.028877,-0.565312,2.694742,7.668724,-0.307374,-0.203515,-0.333919,-0.016997,-0.388660,-0.121127,...,-0.689925,-0.331812,-0.101974,-0.160158,-0.276233,-0.933956,-0.761695,-0.246603,-0.449525,-0.561544
1,-0.826397,-0.258485,-0.030052,0.654784,0.552719,0.315598,-0.333919,-0.016997,-0.388660,-0.121127,...,1.060439,-0.331812,-0.101974,-0.160158,-0.276233,-0.708361,0.816846,-0.246603,0.094053,0.239548
2,-0.612620,-0.565312,0.235780,0.824090,-0.307374,-0.203515,1.400697,-0.016997,-0.128002,-0.121127,...,0.417768,-0.331812,-0.101974,-0.160158,-0.276233,2.237349,-0.496881,0.259188,-0.449525,0.558105
3,-0.418984,2.555542,-0.516106,-0.058210,-0.307374,0.108464,-0.333919,-0.016997,-0.388660,-0.121127,...,-0.210208,-0.217481,-0.101974,-0.160158,-0.276233,-0.798377,0.416617,-0.246603,-0.449525,0.867086
4,-0.119866,-0.435282,-0.516106,-0.368508,-0.307374,-0.203515,-0.333919,-0.016997,0.309336,-0.121127,...,1.263260,-0.331812,-0.101974,-0.160158,-0.276233,0.032049,0.750461,-0.246603,0.894011,-0.561544


In [25]:
#грузим наш последний классификатор
clf_1 = pickle.load(open("clf_stb_may24_2.dat", "rb"))

In [26]:
clf_1

LGBMClassifier(colsample_bytree=1, max_depth=7, n_estimators=200,
               objective='multi:softmax', random_state=0, subsample=0.3)

In [27]:
#предсказываем кластеры
y_pred = clf_1.predict(df_cl, num_iteration=clf_1.best_iteration_)

In [28]:
len(y_pred)

50518

In [29]:
#добавляем столбец с предсказанными кластерами
final_df_sample["clusters"] = y_pred

In [30]:
#проверяем сколько у нас юзеров куда попало. пятого кластера не должно быть, он схлопнут с 4м для лучшей точности предсказаний
#кластер 8 - туда попадают все, кого мы отбраковали по критериям несоответствия своим кластерам
#логично что кластер 7 самый большой после 8го - там те, кто выделяется в корзине хорошим % фруктов, овощей, бакалеи и мяса/птицы, классика
final_df_sample.groupby('clusters').buyer_id.nunique()

clusters
1.0     1818
2.0     2833
3.0      886
4.0     3233
5.0     4489
6.0     5390
7.0     1948
8.0    29921
Name: buyer_id, dtype: int64

In [31]:
#df_pivot = final_df_sample.pivot_table(index='clusters', aggfunc='mean').transpose().reset_index()
#df_pivot.to_excel('check_model.xlsx')

In [32]:
upload_bq = final_df_sample[['buyer_id', 'clusters']]
upload_bq['date'] = last_date
upload_bq.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,buyer_id,clusters,date
0,94578440,8.0,2023-06-01
1,94578475,1.0,2023-06-01
2,94578547,8.0,2023-06-01
3,94578644,8.0,2023-06-01
4,94578760,8.0,2023-06-01


In [ ]:
#загружаем основную таблицу в КХ - пустая табличка уже есть
connection = {
            'host': 'https://clickhouse-02.lenta.tech:8443',
            'password': 'Gnjrw1HhkByi',
            'user': 'a_al_kosheleva',
            'database':'dwh_sandbox'
            }

truncate_table_query = "TRUNCATE TABLE dwh_sandbox.clusters_by_categories"
pandahouse.execute(truncate_table_query, connection=connection)
pandahouse.to_clickhouse(upload_bq, table = "clusters_by_categories", connection=connection, index=False)